<a href="https://colab.research.google.com/github/flying-bear/kompluxternaya/blob/master/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
import nltk
import numpy as np
import re

from collections import Counter
from google.colab import drive
from math import exp, log
from sklearn.base import TransformerMixin

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
root_path = 'gdrive/My Drive/studies/HSE/prog/kompluxternaya'
with open(root_path+'/'+'peace.txt', 'r', encoding='utf-8') as f:
  text = f.read()[2:]
print(len(text))

3227579


In [5]:
def preprocess_text(text):
    # TODO
    # make lowercase
    text = text.lower()
    # replace all punctuation except dots with spaces
    pattern = '\]|!|"|#|\$|%|&|\'|\(|\)|\*|\+|,|-|/|:|;|<|=|>|\?|@|\[|\\|^|_|`|{|\||}|~|”|“|—|‘|’'
    text = re.sub(pattern, ' ', text)
    # collapse multiple spaces into one '   ' -> ' '
    text = re.sub('\s+', ' ', text)
    return text

text = preprocess_text(text)
#assert len(text) == 3141169
len(text)

3141168

In [0]:
text = text.split('.')
text = [x.strip() for x in text]

In [0]:
def list_to_bigrams(l, bigrams=Counter()):
  """
  transforms a list of lists into a counter of their pairs (tuples) across the lists
  l: a list of lists of integers or strings
  """
  for i in range(len(l)):
    bigrams.update((x, y) for x, y in zip(*[l[i][j:] for j in range(2)]))
  return bigrams

In [0]:
def update_token(text, token, token_id):
  """
  modifies text, all instances of the token (element pairs) in text are replaced by token_id
  text: list of lists of integers
  token: integer tuple
  token_id: the id of the given token, integer
  """
  text_new = [inner_list[:] for inner_list in text]
  for i, sent in enumerate(text):
    deletions = 0
    for j, (v, w) in enumerate(zip(sent[:-1], sent[1:])):
      if (v, w) == token:
        text_new[i][j-deletions] = token_id
        del text_new[i][j-deletions+1]
        deletions += 1
  return text_new

In [9]:
t = ['бля бля бл', ' ', 'бля']
test_itos = list(set(''.join(t))) # list letters
test_stoi = dict(zip(test_itos, range(len(test_itos)))) # dict of letters and their ids
t = [[test_stoi[x] for x in t[i]] for i in range(len(t))] # replace letters with thir ids (str to list)
test_vocab_size = 12
while len(test_itos) < test_vocab_size:
    bigrams = list_to_bigrams(t) # get text bigrams
    if bigrams.most_common(1):
      new_token = bigrams.most_common(1)[0][0] # find most common bigram
      new_id = len(test_itos)
      test_itos.append(new_token)
      test_stoi[new_token] = new_id
      # find occurences of the new_token in the text and replace them with new_id
      t = update_token(t, new_token, new_id)
    else:
      break
print(t)
print(test_itos)

[[10, 10, 4], [2], [7]]
['я', 'б', ' ', 'л', (1, 3), (1, 3), (0, 2), (4, 0), (4, 0), (4, 0), (4, 6), (4, 6)]


In [11]:
def recursive_token_lookup(tok, itos): #tok int or tuple
  if type(tok) == int:
    content = itos[tok]
    if type(content) == str:
      return content
    else:
      return recursive_token_lookup(content, itos)
  elif type(tok) == tuple:
    return recursive_token_lookup(tok[0], itos) + recursive_token_lookup(tok[1], itos)
  
recursive_token_lookup(7, test_itos)

'бля'

In [0]:
class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}

    def fit(self, text):
        """
        fits 
        - self.itos (a list of strings (symbols) and integer tuples (most frequent bigrams while vocabulary does not exceed vocab_size
                      each symbol/bigram is indexed by its place in the list) )
        - self.stoi (a dict of symbols and bigrams to thir indicies in the self.stoi list)
        - text (symbols replaced by thir ids, bigrams of ids replaced by their ids
        text: list of strings
        """
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = list(set(''.join(text))) # list letters
        self.stoi = dict(zip(self.itos, range(len(self.itos)))) # dict of letters and their ids
        text = [[self.stoi[x] for x in text[i]] for i in range(len(text))] # replace letters with thir ids (str to list)
        
        while len(self.itos) < self.vocab_size:
            bigrams = list_to_bigrams(text) # get text bigrams
            if bigrams.most_common(1):
              new_token = bigrams.most_common(1)[0][0] # find most common bigram
              new_id = len(self.itos)
              self.itos.append(new_token)
              self.stoi[new_token] = new_id
              # find occurences of the new_token in the text and replace them with new_id
              text = update_token(text, new_token, new_id)
            else:
              break
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of symbol ids then replaces bigrams of ids with their indicies in self.stoi
        text: list of strings
        """
        text_in_vocabulary = [[symbol for symbol in sent if symbol in self.itos] for sent in text] # exclude out of vocabulary symbols
        text = [[self.stoi[letter] for letter in sent] for sent in text_in_vocabulary] # tokenize text by symbols using self.stoi
        for token_id, token in enumerate(self.itos): # find occurences of a complex token in the text and replace them with token_id
            text = update_token(text, token, token_id)    
        return text
    
    def decode_token(self, tok):
        """
        returns a text coded by the tok
        tok: either an int - id, or a tuple - pair of ids)
        """
        def recursive_token_lookup(token): #token int or tuple
          if type(token) == int:
            content = self.itos[token]
            if type(content) == str:
              return content # only returns strings
            else:
              return recursive_token_lookup(content) # continue lookup on the tuple that was found
          elif type(token) == tuple:
            return recursive_token_lookup(token[0]) + recursive_token_lookup(token[1]) # concatenate string results
        return recursive_token_lookup(tok)
            
    def decode(self, text):
        """
        convert token ids into text
        text: tokenized text, list of lists of integers
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [13]:
test_text = ['бля бля бл', ' ', 'бля']
test_vocab_size = 12
test_bpe = BPE(test_vocab_size)
test_tokenized_text = test_bpe.fit_transform(test_text)
test_tokenized_text
test_bpe.decode(test_tokenized_text[0])


'бля бля бл'

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
def list_to_trigrams(l, trigrams=Counter()):
  """
  transforms a list of lists into a counter of their pairs (tuples) across the lists
  l: a list of lists of integers or strings
  """
  for i in range(len(l)):
    trigrams.update((x, y, z) for x, y, z in zip(*[l[i][j:] for j in range(3)]))
  return trigrams

In [0]:
def smooth(count, delta=1, tau=1):
  """
  laplace smoothing
  count: int
  delta: int or float
  tau: int or float
  """
  return (count + delta) ** (1 / tau)

In [17]:
test_counts = list_to_trigrams(test_tokenized_text)
unsmoothed = [test_counts[(1, 0, x)] for x in range(len(set([item for sublist in test_tokenized_text for item in sublist])))]
smoothed = [smooth(test_counts[(1, 0, x)]) for x in range(len(set([item for sublist in test_tokenized_text for item in sublist])))]
print('non-smooth:')
print(unsmoothed)
print('____')
print('smooth:')
print(smoothed)

non-smooth:
[0, 0, 0, 0]
____
smooth:
[1.0, 1.0, 1.0, 1.0]


In [0]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.counts = Counter() # TODO create array for storing 3-gram counts (empty 3-gram counter)

    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        text = [[start_token]+sent+[end_token] for sent in text]
        self.counts = list_to_trigrams(text, self.counts) # count 3-grams in the text  
        return self
        
    def get_proba(self, a, b, c, tau=1):
        """
        get Laplace smoothed probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        trigram = (a, b, c)
        counts_abx = [smooth(self.counts[(a, b, x)]) for x in range(self.vocab_size)] # smooth counts for every possible trigram starting with given bigram
        result = smooth(self.counts[trigram]) / sum(counts_abx) # approximate probability by counters
        return result
            
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab_size for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = [self.get_proba(a,b,x, tau) for x in range(self.vocab_size)]
        return result
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [19]:
l = [[0,1,2,3], [3,1,2,0,4], [4,1,2,3,2,3,1,2,3,2]]
test_lm = LM(len(set([item for sublist in l for item in sublist])), 1).fit(l)
test_lm.infer(1,2)

[0.18181818181818182,
 0.09090909090909091,
 0.09090909090909091,
 0.36363636363636365,
 0.09090909090909091,
 0.09090909090909091,
 0.09090909090909091]

In [35]:
test_l = test_tokenized_text
real_test_lm = LM(len(set([item for sublist in test_l for item in sublist])), 1).fit(test_l)
real_test_lm.infer(3,0)

[0.125, 0.125, 0.375, 0.125, 0.125, 0.125]

In [38]:
lm.infer(1,57)[5:15]

[0.00016960651289009497,
 0.00016960651289009497,
 0.00033921302578018993,
 0.00016960651289009497,
 0.05529172320217096,
 0.018317503392130258,
 0.00016960651289009497,
 0.00016960651289009497,
 0.029511533242876527,
 0.005257801899592945]

In [0]:
def enum_sort_tuple(arr):
  """
  makes list of tuples of ids and their values in the decreasing order of the values from the list input
  arr: flat list
  """
  return sorted(enumerate(arr), key=lambda x:x[1], reverse=True)

In [23]:
enum_sort_tuple([1, 2, 3, 1, 31, 0])

[(4, 31), (2, 3), (1, 2), (0, 1), (3, 1), (5, 0)]

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate prediction list from language model *lm* conditioned on input_seq
    input_seq: list of token ids for conditioning
    lm: language model
    max_len: max generated list length
    k: size of beam
    tau: temperature
    """
    input_seq = [start_token] + input_seq
    x, y = input_seq[-2:]
    beam = [([y, token], proba) for token, proba in enum_sort_tuple(lm.infer(x, y, tau))[:k]] # store in beam tuples of current sequences and their log probabilities
    for i in range(max_len):
      candidates = []
      candidates_log_proba = []
      for snt, snt_log_proba in beam:
          if snt[-1] == end_token:# TODO process terminal token 
            continue
          else:
            a, b = snt[-2:]
            proba = lm.infer(a, b, tau) # probability vector of the next token
            best_k = enum_sort_tuple(proba)[:k] # top-k most probable (token, probability) pairs
            candidates += [snt + [token] for token, proba in best_k] # update candidates' sequences
            candidates_log_proba += [snt_log_proba + log(proba) for token, proba in best_k] # update corresponding probabilities    
      beam = [(candidates[index], log_proba) for index, log_proba in enum_sort_tuple(candidates_log_proba)[:k]] # select top-k most probable sequences from candidates
    return [(snt[1:], exp(snt_log_proba)) for snt, snt_log_proba in beam]

In [25]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for snt, snt_proba in result:
  print(f'hypothesis: "{bpe.decode(snt)}"')
  print(f'probability: {snt_proba}')
    

hypothesis: "with his with hi"
probability: 7.659747275838157e-06
hypothesis: "that it was not "
probability: 5.8625778068365075e-06
hypothesis: "with his with a "
probability: 5.6960700417323595e-06
hypothesis: "that he was now "
probability: 4.65697608804872e-06
hypothesis: "that it was now"
probability: 4.26326906790895e-06
hypothesis: "that he was nown"
probability: 4.042741995970955e-06
hypothesis: "with his with an"
probability: 3.603427822099503e-06
hypothesis: "that he was all "
probability: 3.5799361815179967e-06
hypothesis: "with his with her"
probability: 3.034782911918333e-06
hypothesis: "with his fack "
probability: 3.0185801711278287e-06


In [26]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for snt, snt_proba in result:
  print(f'hypothesis: "{bpe.decode(snt)}"')
  print(f'probability: {snt_proba}')

hypothesis: "ed to his with "
probability: 8.643180715488237e-06
hypothesis: "ed to his with a"
probability: 5.963937713775946e-06
hypothesis: " and with his he"
probability: 5.268217738063301e-06
hypothesis: " and with himper"
probability: 4.874363695393653e-06
hypothesis: "ed to his with h"
probability: 4.748266956771256e-06
hypothesis: " and with his wi"
probability: 4.584553471805025e-06
hypothesis: " and with his fa"
probability: 4.343423544918241e-06
hypothesis: " and with his st"
probability: 4.109638533630364e-06
hypothesis: "ed to his with the "
probability: 3.86630974090511e-06
hypothesis: " and with his a "
probability: 3.826087971181735e-06


In [27]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
for snt, snt_proba in result:
  print(f'hypothesis: "{bpe.decode(snt)}"')
  print(f'probability: {snt_proba}')

hypothesis: " that he was all "
probability: 4.856790571285655e-06
hypothesis: " and with his he"
probability: 4.7857293296851605e-06
hypothesis: " and with himper"
probability: 4.427946311340873e-06
hypothesis: " and with his wi"
probability: 4.1646782848411585e-06
hypothesis: " and with his fa"
probability: 3.945632182203954e-06
hypothesis: " and with his st"
probability: 3.733258312901353e-06
hypothesis: " and with his a "
probability: 3.4756766580364223e-06
hypothesis: " and with himpl"
probability: 3.3557341809769155e-06
hypothesis: " and with his sa"
probability: 3.28192409895358e-06
hypothesis: " and with his for"
probability: 2.995224430286215e-06


In [28]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
for snt, snt_proba in result:
  print(f'hypothesis: "{bpe.decode(snt)}"')
  print(f'probability: {snt_proba}')

hypothesis: "expression of "
probability: 1.1965771057691515e-05
hypothesis: "expressible t"
probability: 9.507365061191772e-06
hypothesis: "expression of the "
probability: 8.647532024886306e-06
hypothesis: "expection of "
probability: 6.809722104443703e-06
hypothesis: "expressible an"
probability: 6.778793922553773e-06
hypothesis: "expressible w"
probability: 6.693526074471334e-06
hypothesis: "expressible s"
probability: 6.082439829880531e-06
hypothesis: "expressible of"
probability: 5.7840023615920045e-06
hypothesis: "expressible b"
probability: 5.7555797455645205e-06
hypothesis: "expection of the "
probability: 4.921311772959316e-06


In [43]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    n = len(snt)
    p = 0
    for trigram in list_to_trigrams([snt]):
      p += log(lm.get_proba(*trigram))
    result = exp(p) ** (-1/n) # perplexity for the sentence
    return result

perplexity(tokenized_text[0], lm)

13.537062655934571